In [11]:
#load neccessary libraries
import pandas as pd
import numpy as np


In [12]:
##linking google sheet 
import gspread 
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('araba-database-b34dc9c828f0.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1IRpTDeZ3XIXqEZaI5JWgJHHEaTWgU9gCi1OxoBUTXRc' 
book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet("Form Responses 1") 
table = worksheet.get_all_values()


In [13]:
##Convert table data into a dataframe 
df = pd.DataFrame(table[1:], columns=table[0])


##Convert number strings to floats and ints 
df = df.apply(pd.to_numeric, errors='ignore')



In [14]:
#Convert date strings to datetime format 
df['Date Of Birth'] = pd.to_datetime(df['Date Of Birth'],infer_datetime_format=True) 
df['Wedding Anniversary (Marrieds only)'] = pd.to_datetime(df['Wedding Anniversary (Marrieds only)'],infer_datetime_format=True)




In [15]:
#df = df.drop(df.columns[[0]], axis=1) 

#join first and last name together 
df['Full Name'] = df['Surname'] + ' '+ df['First Name'] + ' ' + df['Middle Name']

In [17]:
#rearrange the columns
mid = df['Full Name']
df.drop(labels=['Full Name'], axis=1, inplace = True)
df.insert(1, 'Full Name', mid)



In [18]:
#drop  first name and middle name 
#del df['Surname']
df = df.rename(columns = {'Surname': 'Family'})
del df['First Name']
del df['Middle Name']
#capitalise all first letters of names 
df['Full Name']= df['Full Name'].str.title()


In [19]:
df = df.rename(columns = {'Have you been baptized?': 'Baptism'})

In [20]:
# this codes shows duplicated inputs if any 
duplicateDFRow = df[df.duplicated()]
duplicateDFRow

#delete duplicates if any 
df.drop_duplicates(subset ="Full Name", 
                     keep = False, inplace = True) 

In [21]:
#making sure all state of origin inputfollow same format i.e Oyo will be oyo in all cases not Oyo(ibadan) (e get why)
df['State of Origin'] = df['State of Origin'].str.title()
df['State of Origin'] = df[['State of Origin']].applymap(lambda x: x.split()[0]+ ' State')

In [22]:
#lets all column name start with capital letter
for col in df:
    col.title()

#lets all Family name start with capital letter
df['Family']= df['Family'].str.title()



In [23]:
#converts the date of birth and wedding anniversary column from string to datetime format
df['Date Of Birth']= pd.to_datetime(df['Date Of Birth'])
df['Wedding Anniversary (Marrieds only)']= pd.to_datetime(df['Wedding Anniversary (Marrieds only)'])

In [24]:
#this function when ran subtracts todays date from date of birth to give age or years marriage
def from_dob_to_age(born):
    today = datetime.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
def from_yom_to_aom(wed):
    today = datetime.date.today()
    return today.year - wed.year - ((today.month, today.day) < (wed.month, wed.day))

In [25]:
import datetime
#Applying the from_dob_to_age fucntion
df['Age'] = df['Date Of Birth'].apply(lambda x: from_dob_to_age(x))
df['Years of Marriage'] = df['Wedding Anniversary (Marrieds only)'].apply(lambda x: from_dob_to_age(x))

#too many numbers , lets give them their name (eg 1 is to january , 2 is to feburary etc)
import calendar
df['Month'] = pd.DatetimeIndex(df['Date Of Birth']).month
df['Month'] = df['Month'].apply(lambda x: calendar.month_name[x])
df['Month and Day'] = df['Month'].astype(str) +', ' + (pd.DatetimeIndex(df['Date Of Birth']).day ).astype(str)
df['MMonth'] = pd.DatetimeIndex(df['Wedding Anniversary (Marrieds only)']).month
df['MMonth and Day'] = df['MMonth'].astype(str) +', ' + (pd.DatetimeIndex(df['Wedding Anniversary (Marrieds only)']).day ).astype(str)



In [26]:
#this code replaces all blanks into Nan formant 
df.replace(r'^\s*$', np.nan, regex=True,inplace = True)

#replaces any blank in society with non society 
df['Society'].fillna('Non-Society', inplace=True)

In [27]:
#the code below conditions all age into a group 
df["Age"]=df["Age"].astype(int, copy=True)
def age_group(age):
    if age['Age'] <= 12:
        return 'Children'
    elif age['Age'] <= 17:
        return 'Teen' 
    elif age['Age'] <= 45:
        return 'Youth'
    elif age['Age'] <= 70:
        return 'Adult'
    else:
        return 'Elder'

In [28]:
df['AGE GROUP'] = df.apply(age_group, axis=1)

#the code below forces any response other than married or widowed into single
def status(stat):
    if stat['Marital Status'] == 'Single':
        return 'Single'
    elif stat['Marital Status'] == 'Married':
        return 'Married'
    elif stat['Marital Status'] == 'Widowed':
        return 'Widowed'
    else:
        return 'Single'



In [29]:
df['Marital Status'] = df.apply(status, axis=1)

WED = df[['Timestamp','Full Name','Family','Gender','Wedding Anniversary (Marrieds only)','Years of Marriage','MMonth']]
WED = WED.dropna()
WED = WED.astype({"Years of Marriage": int ,'MMonth' : int})

def month(stat):
    if stat['MMonth'] == 1:
        return 'January'
    elif stat['MMonth'] == 2:
        return 'February'
    elif stat['MMonth'] == 3:
        return 'March'
    elif stat['MMonth'] == 4:
        return 'April'
    elif stat['MMonth'] == 5:
        return 'May'
    elif stat['MMonth'] == 6:
        return 'June'
    elif stat['MMonth'] == 7:
        return 'July'
    elif stat['MMonth'] == 8:
        return 'August'
    elif stat['MMonth'] == 9:
        return 'September'
    elif stat['MMonth'] == 10:
        return 'October'
    elif stat['MMonth'] == 11:
        return 'Noevember'
    else:
        return 'December'

In [30]:
WED['MMonth'] = WED.apply(month, axis=1)


WED['MONTH AND DAY'] = WED['MMonth'].astype(str) +', ' + (pd.DatetimeIndex(WED['Wedding Anniversary (Marrieds only)']).day ).astype(str)



In [31]:
#this new dataframe will be sent to our google sheet 
For_Analysis = df[['Timestamp','Full Name','Family','Gender','Date Of Birth','Marital Status','Baptism','Society','AGE GROUP',
                  'State of Origin', 'Local Government of Origin','Wedding Anniversary (Marrieds only)']]
For_Analysis.index.rename('No', inplace=True)

FOR_WEDDINGS = WED[['Timestamp','Full Name','Family','Gender','Wedding Anniversary (Marrieds only)','Years of Marriage','MMonth','MONTH AND DAY']]
FOR_WEDDINGS.index.rename('No', inplace=True)

#this new dataframe will be sent to our google sheet 
contact_details= df[['Timestamp','Full Name','Date Of Birth','Month','Month and Day','Age','Mobile Number','Additional Mobile Number(s)','Residential Address','Gender']]
contact_details.index.rename('No', inplace=True)

In [34]:
# sending contact_details to google sheet
from df2gspread import df2gspread as d2g 
wks_name = 'contact_details' 
d2g.upload(contact_details, spreadsheet_key, wks_name, credentials=credentials, row_names=True)




<Worksheet 'contact_details' id:443988813>

In [35]:
# sending For_Analysis to google sheet
from df2gspread import df2gspread as d2g 
wks_name = 'For_Analysis' 
d2g.upload(For_Analysis, spreadsheet_key, wks_name, credentials=credentials, row_names=True)



<Worksheet 'For_Analysis' id:1799952380>

In [36]:
# sending FOR_WEDDINGS to google sheet
from df2gspread import df2gspread as d2g 
wks_name = 'FOR_WEDDINGS' 
d2g.upload(FOR_WEDDINGS, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

print('code succesfull')

code succesfull
